In [28]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)
from datetime import datetime
import plotly.express as px

In [29]:
# df = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx", sheet_name='出貨明細')

In [30]:
df = pd.read_excel('/Users/kai/OneDrive/INPAQ/業績總表/Weekly report_v0.1.xlsx', sheet_name='出貨明細')

In [31]:
today = datetime.today().strftime('%Y-%m-%d')

In [32]:
filtered = df[(df['Group'] == '華新科技') & (df['狀態'].str.contains('未出')) & (df['預交日期'] >= today) & (df['BU'].isin(['EMC'])) & (df['客戶名稱'] == '華新科技')]

In [33]:
filtered['日數'] = filtered['預交日期'] - filtered['開單日期']

In [34]:
result = filtered [filtered['日數'] >= '84days']


In [35]:
plot = filtered.groupby([pd.Grouper(key='預交日期', freq='M'), '產品分類', '品名'])[['數量']].sum().reset_index()

In [36]:
plot['預交日期'] = plot['預交日期'].dt.strftime('%Y-%m')

In [37]:
plot['數量'] = plot['數量'].astype('int')

In [38]:
chart = plot.style.format({'數量': '{:,.0f}'}).set_caption('華新科EMC產品未交訂單數量').hide_index()

In [39]:
emc = plot.groupby(['品名'])[['數量']].sum().sort_values(by='數量', ascending=False).reset_index()

In [40]:
emc['佔比'] = emc['數量'] / emc['數量'].sum()

In [41]:
plot[plot['預交日期'].isin(['2021-06'])]['數量'].sum()

6692000

In [42]:
# chart.data.to_excel('/Users/kai/Desktop/20210525_未出訂單.xlsx', index=False)

In [43]:
bar_plot = px.bar(plot, x='預交日期', y='數量', color='產品分類', barmode='group', text=plot['數量'])

In [44]:
bar_plot

In [45]:
file_date = datetime.today().strftime('%Y%m%d')

In [46]:
with open('/Users/kai/Desktop/' + file_date + '_未出訂單.html', 'w') as f:
    f.write(chart.render())
    f.write(bar_plot.to_html(full_html=False, include_plotlyjs='cdn') )
    f.close()

In [47]:
result.to_excel('/Users/kai/Desktop/' + file_date + '_超出12週訂單.xlsx', index=False)